# Tables Scratchpad

This notebook is meant for in-house demonstration of candidate extraction and featurization of tables. It assumes an input file in XHTML format, a strict form of HTML that coincides with XML structure, allowing for easy display (HTML) and safe tree traversal (XML).

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of snorkel.parser failed: Traceback (most recent call last):
  File "/Users/bradenhancock/anaconda/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'HTMLOmniParser' is not defined
]


### Candidate Extraction

First, import the 'HTMLParser' class to read HTML tables

In [8]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/diseases/diseases.xhtml')

The "TableParser" class divides the html doc into cells, adding a 'cell_id' attribute to each cell for future traversal, and creating "Cell" objects that have attributes such as row number, column number, html tag, html attributes, and any tags/attributes on a cells ancestors in the table.

In [9]:
from snorkel.parser import TableParser
table_parser = TableParser()

As usual, pass these to a Corpus object for digestion.

In [10]:
# from snorkel.parser import Corpus
# %time corpus = Corpus(html_parser, table_parser)

from snorkel.parser import CorpusParser
cp = CorpusParser(html_parser, table_parser)
%time corpus = cp.parse_corpus(name='Diseases Corpus')

CPU times: user 78.8 ms, sys: 12.2 ms, total: 91 ms
Wall time: 118 ms


In [11]:
print len(corpus.documents)
print len(corpus.documents[0].tables)
print len(corpus.documents[0].cells)
print len(corpus.documents[0].phrases)
print corpus.documents[0].tables[0].cells[0].phrases[0].text

1
2
24
25
Disease


In [12]:
doc = corpus.documents[0]
for phrase in doc.phrases: print phrase

Phrase('0', 0, 0, 0, u'Disease')
Phrase('0', 0, 1, 0, u'Location')
Phrase('0', 0, 2, 0, u'Year')
Phrase('0', 0, 3, 0, u'Polio')
Phrase('0', 0, 4, 0, u'New York')
Phrase('0', 0, 5, 0, u'1914')
Phrase('0', 0, 6, 0, u"I don't like Chicken Pox.")
Phrase('0', 0, 6, 1, u'The plague is also bad.')
Phrase('0', 0, 7, 0, u'Boston')
Phrase('0', 0, 8, 0, u'2001')
Phrase('0', 0, 9, 0, u'Scurvy')
Phrase('0', 0, 10, 0, u'Annapolis')
Phrase('0', 0, 11, 0, u'1901')
Phrase('0', 1, 0, 0, u'Problem')
Phrase('0', 1, 1, 0, u'Cause')
Phrase('0', 1, 2, 0, u'Cost')
Phrase('0', 1, 3, 0, u'Arthritis')
Phrase('0', 1, 4, 0, u'Pokemon Go')
Phrase('0', 1, 5, 0, u'Free')
Phrase('0', 1, 6, 0, u'Yellow Fever')
Phrase('0', 1, 7, 0, u'Unicorns')
Phrase('0', 1, 8, 0, u'$17.75')
Phrase('0', 1, 9, 0, u'Hypochondria')
Phrase('0', 1, 10, 0, u'Fear')
Phrase('0', 1, 11, 0, u'$100')


Load the good 'ole disease dictionary for recognizing disease names.

In [ ]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Here we use a new CandidateSpace object, CellNgrams. It inherits from Ngrams, and ensures that the Table context object is broken up into cells before being passed into the usual routine for pulling out Ngrams.

In [ ]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
table_ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Passing the CandidateSpace, Matcher, and Context objects to a Candidates object, extraction is performed, and we see that a number of disease CellNgrams are returned.

In [ ]:
# With new Candidates object:
# from snorkel.candidates import Candidates
# %time candidates = Candidates(table_ngrams, disease_matcher, corpus.get_contexts())

# With old Candidates object:
from snorkel.candidates import EntityExtractor
ce = EntityExtractor(table_ngrams, disease_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')

for cand in candidates: print cand

In [ ]:
c = candidates[0]
for ngram in c.row_ngrams('words'): print ngram

### Feature Generation

We can then generate features on our set of candidates, including *new and improved* table features!

In [ ]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer()
featurizer.fit_transform(candidates)

In [ ]:
featurizer.get_features_by_candidate(candidates[1])[:]

Ta-da! Next up: feeding these features into the learning machine.